## Download varidal files

In [0]:
!wget https://raw.githubusercontent.com/ftyers/vardial-shared-task/master/test/roa-covered

In [0]:
!wget https://raw.githubusercontent.com/ftyers/vardial-shared-task/master/train/roa-uncovered

## Download transliterated files

In [0]:
!wget https://raw.githubusercontent.com/NIS-2018-CROSS-M/vardial-shared-task/master/train/roa-uncovered-transliterated

In [0]:
!wget https://raw.githubusercontent.com/NIS-2018-CROSS-M/vardial-shared-task/master/test/roa-covered-transliterated

In [0]:
!head roa-uncovered

In [0]:
!head roa-uncovered-transliterated

In [0]:
!head roa-covered

In [0]:
!head roa-covered-transliterated

## Prepare train and test files for morfessor within 5 folds

In [0]:
# wordforms extraction
!cut -f2 roa-covered-t > roa-cov

### fold 1
#### train = head 0.8n; test = tail 0.2n

In [0]:
# train
!head -$(cat roa-cov | python -c 'import sys; print(round(len([line.strip() for line \
in sys.stdin]) * 0.8))') roa-cov > train1.txt
!wc -l train1.txt
# test
!tail -$(cat roa-cov | python -c 'import sys; print(round(len([line.strip() for line \
in sys.stdin]) * 0.2))') roa-cov > test1.txt
!wc -l test1.txt
# morfessor
!morfessor -t train1.txt -T test1.txt > srd_segmented.txt

### fold 2
#### train = head 0.75n train1 + test1; test = tail 0.25n train1

In [0]:
# train
!head -$(cat train1.txt | python -c 'import sys; print(round(len([line.strip() for line \
in sys.stdin]) * 0.75))') train1.txt >> train2.txt && cat test1.txt >> train2.txt
!wc -l train2.txt
# test
!tail -$(cat train1.txt | python -c 'import sys; print(round(len([line.strip() for line \
in sys.stdin]) * 0.25))') train1.txt > test2.txt
!wc -l test2.txt
# morfessor
!morfessor -t train1.txt -T test1.txt >> srd_segmented.txt
!wc -l srd_segmented.txt

### fold 3
#### train = head 0.5n train1 + test1 + test2; test = tail 0.3n train1

In [0]:
# train
!head -$(cat train1.txt | python -c 'import sys; print(round(len([line.strip() for line \
in sys.stdin]) * 0.5 - 1))') train1.txt >> train3.txt && cat test1.txt >> train3.txt && cat test2.txt >> train3.txt
!wc -l train3.txt
# test
!head -$(cat train1.txt | python -c 'import sys; print(round(len([line.strip() for line \
in sys.stdin]) * 0.75))') train1.txt | tail -$(wc -l test1.txt | cut -f1 -d' ') > test3.txt
!wc -l test3.txt
# morfessor
!morfessor -t train3.txt -T test3.txt >> srd_segmented.txt
!wc -l srd_segmented.txt

### fold 4
#### train =  tail 0.8n; test = head 0.2n

In [0]:
# train
!tail -$(cat roa-cov | python -c 'import sys; print(round(len([line.strip() for line \
in sys.stdin]) * 0.8))') roa-cov > train4.txt
!wc -l train4.txt
# test
!head -$(cat roa-cov | python -c 'import sys; print(round(len([line.strip() for line \
in sys.stdin]) * 0.2))') roa-cov > test4.txt
!wc -l test4.txt
# morfessor
!morfessor -t train4.txt -T test4.txt >> srd_segmented.txt
!wc -l srd_segmented.txt

### fold 5
#### train = tail 0.75n train4 + test4; test = head train4

In [0]:
# train
!tail -$(cat train4.txt | python -c 'import sys; print(round(len([line.strip() for line \
in sys.stdin]) * 0.75))') train4.txt >> train5.txt && cat test4.txt >> train5.txt
!wc -l train5.txt
# test
!head -$(cat train4.txt | python -c 'import sys; print(round(len([line.strip() for line \
in sys.stdin]) * 0.25))') train4.txt > test5.txt
!wc -l test5.txt
# morfessor
!morfessor -t train5.txt -T test5.txt >> srd_segmented.txt

### Download morfessor segmentation file if needed

In [0]:
!wget https://www.dropbox.com/s/wu4ojsj18il5bcf/srd_segmented.txt

In [10]:
!wc -l srd_segmented.txt

9900 srd_segmented.txt


In [11]:
!head srd_segmented.txt

обур ланы
област ына
област ланы
о аз и с ле
нюзюр леринде
нюзюр лери
нохта гъа
норма ланы
норма гъа
номер инден


# Tag projection

## roa and srd intersection

###  Creating roa dictionary; key:value  == wordform:[lemma, pos, analysis]

In [12]:
from collections import defaultdict as dd


roa_keys = [line.strip().split('\t')[1] for line in open('roa-uncovered-transliterated', 'r', encoding='utf-8') if line]
roa_values = [line.strip().split('\t')[2:] for line in open('roa-uncovered-transliterated', 'r', encoding='utf-8') if line]
roa = dict(zip(roa_keys, roa_values))

roa_intersection = dd(set)

for line in open('roa-uncovered-transliterated', 'r'):
  iso, wf, lem, pos, analysis = line.rstrip().split('\t')
  roa_intersection[wf].add((lem, pos, analysis))

print('roa:', list(roa.items())[:5])
print('roa_intersection:', list(roa_intersection.items())[:5])

roa: [('казак', ['казак', 'NOUN', 'Case=Nom']), ('муамеле', ['муамеле', 'NOUN', 'Case=Nom']), ('заманы', ['заман', 'NOUN', 'Case=Nom|Number[psor]=Sing,Plur|Person[psor]=3']), ('конут', ['конут', 'NOUN', 'Case=Nom']), ('корумасы', ['корума', 'NOUN', 'Case=Nom|Number[psor]=Sing,Plur|Person[psor]=3'])]
roa_intersection: [('казак', {('казак', 'ADJ', '_'), ('казак', 'NOUN', 'Case=Nom')}), ('муамеле', {('муамеле', 'NOUN', 'Case=Nom')}), ('заманы', {('заман', 'NOUN', 'Case=Acc'), ('заман', 'NOUN', 'Case=Nom|Number[psor]=Sing,Plur|Person[psor]=3')}), ('конут', {('конут', 'NOUN', 'Case=Nom')}), ('корумасы', {('корума', 'NOUN', 'Case=Nom|Number[psor]=Sing,Plur|Person[psor]=3'), ('кору', 'VERB', 'Case=Nom|Number[psor]=Sing,Plur|Person[psor]=3|Valency=2|VerbForm=Vnoun')})]


### Creating srd dictionary; wf_roa[i] tags are projected on wf_srd[i] within the intersection

In [137]:
srd_words = [line.strip().split('\t')[1] for line in open('roa-covered-transliterated', 'r', encoding='utf-8')]
srd = {wordform: roa_intersection[wordform] for wordform in srd_words if wordform in roa_intersection.keys()}
len(srd.keys())

1763

In [14]:
list(srd.items())[:5]

[('крал', ['крал', 'NOUN', 'Case=Nom']),
 ('алай', ['алай', 'NOUN', 'Case=Nom']),
 ('баш', ['баш', 'NOUN', 'Case=Nom']),
 ('адам', ['адам', 'NOUN', 'Case=Nom']),
 ('барды',
  ['бар',
   'VERB',
   'Aspect=Perf|Number=Sing|Person=3|Tense=Past|Valency=1|VerbForm=Fin'])]

Reading morfessor file as a list

In [0]:
morfessor = [line.strip().replace('гъ', 'г').replace('къ', 'к') for line in open('srd_segmented.txt', 'r', encoding='utf-8')]

In [16]:
morfessor[:5]

['обур ланы', 'област ына', 'област ланы', 'о аз и с ле', 'нюзюр леринде']

We consider the substring of index 0 as a morfessor lemma, e.g. in музыка да the lem_morf is музыка

In [0]:
morf_lemmas = [wordform.split()[0] for wordform in morfessor]

## NOUN cluster

We consider the srd wordforms to be in one noun cluster if wf_srd[i] == lem_roa[i] and w_roa[i] POS is NOUN 

In [18]:
srd_noun_lemmas = [wf for wf in srd.keys() if wf == roa[wf][0] and roa[wf][1] == 'NOUN']
len(srd_noun_lemmas)

912

Finding intersection of lemmas_srd and lemmas_morf


In [19]:
lemma_noun_intersection = set(srd_noun_lemmas) & set(morf_lemmas)
len(lemma_noun_intersection)

477

In [20]:
'''если найденная морфессором лемма совпадает с леммой из intersection:
– создаем новое подмножество inter_morf_seg, где хранятся сегментированные строки (e.g. музыка да)
– создаем список неуникальных лемм таких, что лемма соответствует сегментированной строке из inter_morf_seg такой, что 
сегментированная строка:лемма (e.g. музыка да:музыка)'''

noun_segment = [segmented for segmented in morfessor for lemma in lemma_noun_intersection \
                if lemma == segmented.split()[0]]
noun_segment[:5]

['норма ланы', 'норма га', 'низам лары', 'низам ланы', 'низам ла']

In [141]:
'''если склеенная сегментированная строка[i] (=словоформа А) совпадает со словоформой В из множества
roa-uncovered и если лемма[i] совпадает с леммой_[i] из roa-uncovered, мы проецируем теги
словоформы В на словоформу А. создаем список кортежей таких, что:
- projections[0] == словоформа А
- projections[1] == лемма и проецируемые pos и analysis'''

noun_projections = [(segmented, roa[wordform]) for segmented in noun_segment for wordform in \
              roa.keys() if ''.join(segmented.split()) == wordform and segmented.split()[0] \
              == roa[wordform][0] and roa[wordform][1] == 'NOUN']
noun_projections[:5]

[('ун', ['ун', 'NOUN', 'Case=Nom']),
 ('ине', ['ине', 'NOUN', 'Case=Nom']),
 ('га', ['га', 'NOUN', 'Case=Nom']),
 ('ден', ['ден', 'NOUN', 'Case=Nom']),
 ('дан', ['дан', 'NOUN', 'Case=Nom'])]

In [22]:
'''создаем карту тегов на материале существительных из 20% слов srd_words
при условии, что analysis != Case=Nom и морфема встречается 2 и более раза. остальные
либо спорные и нужно смотреть на следующих фолдах, либо склеенные, как сында'''

noun_morphemes = [projection[0].split()[1] for projection in noun_projections if \
                  ''.join(projection[0].split()) != projection[1][0]]
noun_m_freq = {morpheme: noun_morphemes.count(morpheme) for morpheme in noun_morphemes}
list(noun_m_freq.items())[:5]

[('да', 26), ('ы', 37), ('лары', 34), ('сы', 41), ('ун', 7)]

In [23]:
noun_morphs = {projection[0].split()[1]: projection[1][2] for projection in noun_projections if projection[1][1] \
         == 'NOUN' and ''.join(projection[0].split()) != projection[1][0]}

morphemes_2_tagsets = {key: set(value.split('|')) for key, value in noun_morphs.items()}

inflected_morphemes_2_possible_tagsets = {}

for morpheme1 in morphemes_2_tagsets.keys():
    for morpheme2 in morphemes_2_tagsets.keys():

        if morpheme1 == morpheme2 or not morpheme2.endswith(morpheme1):
            continue

        morpheme1_in_morpheme2_idx = morpheme2.find(morpheme1)
        new_morpheme = morpheme2[:morpheme1_in_morpheme2_idx]

        if len(new_morpheme) > 1:
            new_morpheme_tagset = morphemes_2_tagsets[morpheme2] - morphemes_2_tagsets[morpheme1]

            if new_morpheme not in inflected_morphemes_2_possible_tagsets.keys():
                inflected_morphemes_2_possible_tagsets[new_morpheme] = []
            inflected_morphemes_2_possible_tagsets[new_morpheme].append(new_morpheme_tagset)


result_morphemes_2_possible_tagsets = {key: [value] for key, value in morphemes_2_tagsets.items()}
for morpheme, possible_tagsets in inflected_morphemes_2_possible_tagsets.items():
    for possible_tagset in possible_tagsets:

        if morpheme not in result_morphemes_2_possible_tagsets.keys():
            result_morphemes_2_possible_tagsets[morpheme] = [possible_tagset]

        known_tagsets_of_morpheme = result_morphemes_2_possible_tagsets[morpheme]

        if not all(tag in known_tagset for tag in possible_tagset for known_tagset in known_tagsets_of_morpheme):
            result_morphemes_2_possible_tagsets[morpheme].append(possible_tagset)

for morpheme, tagsets in result_morphemes_2_possible_tagsets.items():
    if len(tagsets) > 1:
      if len(tagsets) > 1:
        if morpheme == 'ын':
          del tagsets[0] 
        else:
          del tagsets[1]
          
noun_segment = {morpheme: known_tagsets[0]
                              for morpheme, known_tagsets in result_morphemes_2_possible_tagsets.items()}
list(noun_segment.items())[:5]

[('да', {'Case=Loc'}),
 ('ы', {'Case=Nom', 'Number[psor]=Sing,Plur', 'Person[psor]=3'}),
 ('лары',
  {'Case=Nom', 'Number=Plur', 'Number[psor]=Sing,Plur', 'Person[psor]=3'}),
 ('сы', {'Case=Nom', 'Number[psor]=Sing,Plur', 'Person[psor]=3'}),
 ('ун', {'Case=Acc', 'Number[psor]=Sing,Plur', 'Person[psor]=3'})]

## VERB cluster

We consider the srd wordforms to be in one noun cluster if wf_srd[i] == lem_roa[i] and w_roa[i] POS is VERB

In [24]:
srd_verb_lemmas = [wf for wf in srd.keys() if wf == roa[wf][0] and roa[wf][1] == 'VERB']
len(srd_verb_lemmas)

39

In [25]:
lemma_verb_intersection = set(srd_verb_lemmas) & set(morf_lemmas)
len(lemma_verb_intersection)

32

In [26]:
verb_segment = [segmented for segmented in morfessor for lemma in lemma_verb_intersection \
                if lemma == segmented.split()[0]]
verb_segment[:5]

['кыз лары', 'кыз га', 'кул лугу ндан', 'кул лугу н', 'кул ак лары']

In [27]:
verb_projections = [(segmented, roa[wordform]) for segmented in verb_segment for wordform in \
              roa.keys() if ''.join(segmented.split()) == wordform and segmented.split()[0] \
              == roa[wordform][0] and roa[wordform][1] == 'VERB']
verb_projections[:5]

[('кара р',
  ['кара', 'VERB', 'Number=Sing|Person=3|Tense=Aor|Valency=1|VerbForm=Fin']),
 ('кара п', ['кара', 'VERB', 'Aspect=Perf|Valency=1|VerbForm=Conv']),
 ('кир',
  ['кир', 'VERB', 'Mood=Imp|Number=Sing|Person=2|Valency=1|VerbForm=Fin']),
 ('де с ек',
  ['де', 'VERB', 'Mood=Cond|Number=Plur|Person=1|Valency=2|VerbForm=Conv']),
 ('ат ылган',
  ['ат',
   'VERB',
   'Number=Sing|Person=3|Tense=Past|Valency=2|VerbForm=Fin|Voice=Pass'])]

In [28]:
verb_morphemes = [''.join(projection[0].split()[1:]) for projection in verb_projections if ''.join(projection[0].split()) != projection[1][0]]
verb_m_freq = {morpheme: verb_morphemes.count(morpheme) for morpheme in verb_morphemes}
list(verb_m_freq.items())[:5]

[('р', 2), ('п', 3), ('сек', 2), ('ылган', 2), ('ип', 1)]

In [29]:
verb_morphs = {projection[0].split()[1]: projection[1][2] for projection in verb_projections if projection[1][1] \
         == 'VERB' and ''.join(projection[0].split()) != projection[1][0]}

morphemes_2_tagsets = {key: set(value.split('|')) for key, value in verb_morphs.items()}

inflected_morphemes_2_possible_tagsets = {}

for morpheme1 in morphemes_2_tagsets.keys():
    for morpheme2 in morphemes_2_tagsets.keys():

        if morpheme1 == morpheme2 or not morpheme2.endswith(morpheme1):
            continue

        morpheme1_in_morpheme2_idx = morpheme2.find(morpheme1)
        new_morpheme = morpheme2[:morpheme1_in_morpheme2_idx]

        if len(new_morpheme) > 1:
            new_morpheme_tagset = morphemes_2_tagsets[morpheme2] - morphemes_2_tagsets[morpheme1]

            if new_morpheme not in inflected_morphemes_2_possible_tagsets.keys():
                inflected_morphemes_2_possible_tagsets[new_morpheme] = []
            inflected_morphemes_2_possible_tagsets[new_morpheme].append(new_morpheme_tagset)


result_morphemes_2_possible_tagsets = {key: [value] for key, value in morphemes_2_tagsets.items()}
for morpheme, possible_tagsets in inflected_morphemes_2_possible_tagsets.items():
    for possible_tagset in possible_tagsets:

        if morpheme not in result_morphemes_2_possible_tagsets.keys():
            result_morphemes_2_possible_tagsets[morpheme] = [possible_tagset]

        known_tagsets_of_morpheme = result_morphemes_2_possible_tagsets[morpheme]

        if not all(tag in known_tagset for tag in possible_tagset for known_tagset in known_tagsets_of_morpheme):
            result_morphemes_2_possible_tagsets[morpheme].append(possible_tagset)

for morpheme, tagsets in result_morphemes_2_possible_tagsets.items():
    if len(tagsets) > 1:
        print(f"error! morpheme {morpheme} relates to more than one possible tags string")

verb_segment = {morpheme: known_tagsets[0]
                              for morpheme, known_tagsets in result_morphemes_2_possible_tagsets.items()}

list(verb_segment.items())[:5]

[('р', {'Number=Sing', 'Person=3', 'Tense=Aor', 'Valency=1', 'VerbForm=Fin'}),
 ('п', {'Aspect=Perf', 'Valency=2', 'VerbForm=Conv'}),
 ('с', {'Mood=Cond', 'Number=Plur', 'Person=1', 'Valency=2', 'VerbForm=Conv'}),
 ('ылган',
  {'Number=Sing',
   'Person=3',
   'Tense=Past',
   'Valency=2',
   'VerbForm=Fin',
   'Voice=Pass'}),
 ('се',
  {'Mood=Cond', 'Number=Plur', 'Person=3', 'Valency=1', 'VerbForm=Conv'})]

# Calma POS and my greedy segmentation

In [0]:
!wget https://www.dropbox.com/s/2rkq5cw0vnq52a9/calma_pos.nosys

In [31]:
!head calma_pos.nosys

srd	джыл	cc	NOUN	Tag1=Value1|Tag2=Value2
srd	джыл	cc	ADJ	Tag1=Value1|Tag2=Value2
srd	эм	cc	NOUN	Tag1=Value1|Tag2=Value2
srd	эм	cc	VERB	Tag1=Value1|Tag2=Value2
srd	джылны	cc	NOUN	Tag1=Value1|Tag2=Value2
srd	болганды	cc	VERB	Tag1=Value1|Tag2=Value2
srd	дери	cc	NOUN	Tag1=Value1|Tag2=Value2
srd	кёре	cc	NOUN	Tag1=Value1|Tag2=Value2
srd	кёре	cc	VERB	Tag1=Value1|Tag2=Value2
srd	сора	cc	NOUN	Tag1=Value1|Tag2=Value2


In [32]:
import pandas as pd


calma = pd.read_csv('calma_pos.nosys', sep='\t', names=['iso', 'transliterated', 'lemma', 'POS', 'analysis'], index_col=False)
calma.insert(1, 'wordform', calma['transliterated'])

calma.head()

,iso,wordform,transliterated,lemma,POS,analysis
0,srd,джыл,джыл,cc,NOUN,Tag1=Value1|Tag2=Value2
1,srd,джыл,джыл,cc,ADJ,Tag1=Value1|Tag2=Value2
2,srd,эм,эм,cc,NOUN,Tag1=Value1|Tag2=Value2
3,srd,эм,эм,cc,VERB,Tag1=Value1|Tag2=Value2
4,srd,джылны,джылны,cc,NOUN,Tag1=Value1|Tag2=Value2


In [33]:
# creating three calma pos clusters

clusters, wordforms = list(calma.groupby(['POS']).groups.keys()), []

for cluster in clusters:
  wordforms.append(calma.loc[calma.groupby(['POS']).groups[cluster],
                            'transliterated'].tolist())
  
calma_pos = dict(zip(clusters, wordforms))

for cluster in calma_pos.keys():
  print((cluster, len(calma_pos[cluster])))

('ADJ', 1042)
('ADV', 105)
('NOUN', 8818)
('VERB', 2963)


In [58]:
# NOUN

input_wfs_set = set()
greed_nouns = {}

for noun in calma_pos['NOUN']:
  input_wfs_set.add(noun)
  
processed_wfs_set = set()
result_n_data = {}
for morpheme in sorted(noun_segment.keys(), key=lambda word: len(word), reverse=True):
    processed_wfs_set = set()
    for wf in input_wfs_set:
        if wf.endswith(morpheme):

            if wf != morpheme:
                lemma = wf[:wf.rfind(morpheme)]
            else:
                lemma = "NONE"

            result_n_data[wf] = [lemma] + ['NOUN'] + list(noun_segment[morpheme])
            processed_wfs_set.add(wf)
    input_wfs_set -= processed_wfs_set

    
for morpheme, inflected_data in result_n_data.items():
    lemma = inflected_data[0]
    pos = inflected_data[1]
    analysis = sorted(inflected_data[2:])
    greed_nouns[morpheme] = [lemma, pos, '|'.join(analysis)]
    
for wf in input_wfs_set:
    greed_nouns[wf] = [wf, 'NOUN', "Case=Nom"]
                           
len(list(greed_nouns.keys()))

7732

In [59]:
list(greed_nouns.items())[:2]

[('орунларындан',
  ['орун',
   'NOUN',
   'Case=Abl|Number=Plur|Number[psor]=Sing,Plur|Person[psor]=3']),
 ('чурумларындан',
  ['чурум',
   'NOUN',
   'Case=Abl|Number=Plur|Number[psor]=Sing,Plur|Person[psor]=3'])]

In [73]:
# VERB

input_wfs_set = set()
greed_verbs = {}

for verb in calma_pos['VERB']:
  input_wfs_set.add(verb)

processed_wfs_set = set()
result_v_data = {}
for morpheme in sorted(verb_segment.keys(), key=lambda word: len(word), reverse=True):
    processed_wfs_set = set()
    for wf in input_wfs_set:
        if wf.endswith(morpheme):

            if wf != morpheme:
                lemma = wf[:wf.rfind(morpheme)]
            else:
                lemma = "NONE"

            result_v_data[wf] = [lemma] + ['VERB'] + list(verb_segment[morpheme])
            processed_wfs_set.add(wf)
    input_wfs_set -= processed_wfs_set

    
for morpheme, inflected_data in result_v_data.items():
    lemma = inflected_data[0]
    pos = inflected_data[1]
    analysis = sorted(inflected_data[2:])
    greed_verbs[morpheme] = [lemma, pos, '|'.join(analysis)]
    
for wf in input_wfs_set:
    greed_verbs[wf] = ['bad_verb', 'bad_verb', 'bad_verb']
                           
len(list(greed_verbs.keys()))

2709

In [74]:
list(greed_verbs.items())[:2]

[('сыйырылган',
  ['сыйыр',
   'VERB',
   'Number=Sing|Person=3|Tense=Past|Valency=2|VerbForm=Fin|Voice=Pass']),
 ('чыгарылган',
  ['чыгар',
   'VERB',
   'Number=Sing|Person=3|Tense=Past|Valency=2|VerbForm=Fin|Voice=Pass'])]

In [0]:
!wget https://www.dropbox.com/s/roqm9n2ow9lfzh6/tmp.txt

In [129]:
!head tmp.txt

srd	эм	эм	NOUN	Case=Nom
srd	болганды	бол	VERB	Aspect=Perf|Number=Sing|Person=3|Tense=Past|Valency=1|VerbForm=Fin
srd	болганды	болган	NOUN	Case=Acc
srd	болганды	болганды	ADJ	_
srd	кёре	кёр	VERB	Aspect=Imp|Valency=2|VerbForm=Conv
srd	кёре	кёре	ADJ	_
srd	кёре	кёре	NOUN	Case=Nom
srd	болады	бол	VERB	Number=Plur|Person=3|Tense=Aor|Valency=1|VerbForm=Fin
srd	болады	бол	VERB	Number=Sing|Person=3|Tense=Aor|Valency=1|VerbForm=Fin
srd	болады	бол	VERB	Number=Sing|Person=3|Tense=Aor|Valency=2|VerbForm=Fin


In [0]:
my_calma = []

unique_nouns, unique_verbs  = set(greed_nouns.keys()), set(greed_verbs.keys())
unique_advs, unique_adjs = set(calma_pos['ADV']), set(calma_pos['ADJ'])

    
with open('calma_pos.nosys', 'r') as f:
  for line in f:
    iso, wf, lemma, pos, analysis = line.strip().split('\t')
    if wf in unique_nouns and pos == greed_nouns[wf][1]:
      lemma = greed_nouns[wf][0]
      analysis = greed_nouns[wf][2]
    else:
      lemma = 'bad_noun'
      analysis = 'bad_noun'
      if wf in unique_verbs and pos == greed_verbs[wf][1]:
        lemma = greed_verbs[wf][0]
        analysis = greed_verbs[wf][2]
      else:
        lemma = 'bad_verb'
        analysis = 'bad_verb'
        if wf in unique_advs and pos == 'ADV':
          lemma = wf
          analysis = '_'
        elif wf in unique_adjs and pos == 'ADJ':
          lemma = wf
          analysis = '_'
    my_calma.append((iso, wf, lemma, pos, analysis))

In [0]:
calma_list = []

unique_nouns, unique_verbs  = set(greed_nouns.keys()), set(greed_verbs.keys())
unique_advs, unique_adjs = set(calma_pos['ADV']), set(calma_pos['ADJ'])


with open('calma_pos.nosys', 'r') as f:
  for line in f:
    iso, wf, lemma, pos, analysis = line.strip().split('\t')
    if wf == 'ырдайт':
      print(wf)
    if wf in unique_nouns and pos == greed_nouns[wf][1]:
      lemma = greed_nouns[wf][0]
      analysis = greed_nouns[wf][2]
    else:
      lemma = 'bad_noun'
      analysis = 'bad_noun'
      if wf in unique_verbs and pos == greed_verbs[wf][1]:
        lemma = greed_verbs[wf][0]
        analysis = greed_verbs[wf][2]
      else:
        lemma = 'bad_verb'
        analysis = 'bad_verb'
        if wf in unique_advs and pos == 'ADV':
          lemma = wf
          analysis = '_'
        elif wf in unique_adjs and pos == 'ADJ':
          lemma = wf
          analysis = '_'
    results=[]
    
    if wf in roa_intersection.keys() and wf in srd_words:
      for roa_value in roa_intersection[wf]:
        if pos == roa_value[1]:
          results.append((roa_value[0], roa_value[2]))
    if len(results) == 0:
      results.append((lemma, analysis))
    for result in results:
      calma_list.append((iso, wf, result[0], pos, result[1]))

In [155]:
len(calma_list)

13642

In [0]:
calm = pd.DataFrame(calma_list, columns=['iso', 'wf', 'lemma', 'pos', 'tags'])

In [157]:
calm.head()

,iso,wf,lemma,pos,tags
0,srd,джыл,джыл,NOUN,Case=Nom
1,srd,джыл,джыл,ADJ,_
2,srd,эм,эм,NOUN,Case=Nom
3,srd,эм,bad_verb,VERB,bad_verb
4,srd,джылны,джыл,NOUN,Case=Acc


In [158]:
calm.tags.value_counts()[:5]

Case=Nom                                          3014
bad_verb                                          1718
Case=Acc                                          1554
Case=Nom|Number[psor]=Sing,Plur|Person[psor]=3    1146
_                                                 1145
Name: tags, dtype: int64

In [159]:
tmp = pd.read_csv('tmp.txt', sep='\t', names=['iso', 'wf', 'lemma', 'pos', 'tags'])
tmp.head()

,iso,wf,lemma,pos,tags
0,srd,эм,эм,NOUN,Case=Nom
1,srd,болганды,бол,VERB,Aspect=Perf|Number=Sing|Person=3|Tense=Past|Va...
2,srd,болганды,болган,NOUN,Case=Acc
3,srd,болганды,болганды,ADJ,_
4,srd,кёре,кёр,VERB,Aspect=Imp|Valency=2|VerbForm=Conv


In [160]:
calm = calm[calm.tags != 'bad_verb']
calm.tags.value_counts()[:5]

Case=Nom                                          3014
Case=Acc                                          1554
Case=Nom|Number[psor]=Sing,Plur|Person[psor]=3    1146
_                                                 1145
Case=Loc                                           930
Name: tags, dtype: int64

In [161]:
my_model_predictions = pd.concat([calm, tmp])
my_model_predictions.tags.value_counts()[:5]

Case=Nom                                          3536
Case=Acc                                          1788
_                                                 1307
Case=Nom|Number[psor]=Sing,Plur|Person[psor]=3    1255
Case=Loc                                          1199
Name: tags, dtype: int64

In [162]:
my_model_predictions.head()

,iso,wf,lemma,pos,tags
0,srd,джыл,джыл,NOUN,Case=Nom
1,srd,джыл,джыл,ADJ,_
2,srd,эм,эм,NOUN,Case=Nom
4,srd,джылны,джыл,NOUN,Case=Acc
6,srd,дери,дер,NOUN,"Case=Nom|Number[psor]=Sing,Plur|Person[psor]=3"


In [0]:
my_model_predictions.to_csv('spectie.txt', sep='\t', header=False, index=False, encoding='utf-8')
files.download('spectie.txt')

In [0]:
srd_transliteration = dict(zip(srd_words, [line.strip().split('\t')[1] for line in open('roa-covered', 'r')]))

In [0]:
for non_tr in srd_transliteration.keys():
  my_model_predictions['wf'] = my_model_predictions.wf.str.replace(non_tr, srd_transliteration[non_tr])


In [0]:
import re


def clean(text):
  text = re.sub('ъ+', 'ъ', text)
  return text
  
my_model_predictions['wf'] = my_model_predictions['wf'].apply(clean)

In [0]:
def end_word(word):
  word = re.sub('гъ$', 'къ', word)
  word = re.sub('къ$', 'къ', word)
  return word

In [188]:
end_word('агъымлаг')

'агъымлаг'